In [74]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

from tqdm import tqdm_notebook
import lightgbm as lgb
from catboost import Pool, CatBoostClassifier
import xgboost as xgb
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings("ignore")

import seaborn as sns



plt.style.use('seaborn')
sns.set(font_scale=1)

import gc

In [75]:
train = pd.read_csv("../input/landslide-prevention-and-innovation-challenge/Train (12).csv")
test = pd.read_csv("../input/landslide-prevention-and-innovation-challenge/Test (13).csv")
sample = pd.read_csv('../input/landslide-prevention-and-innovation-challenge/SampleSubmission (7).csv')

In [76]:
train["Label"].value_counts()

0    8148
1    2716
Name: Label, dtype: int64

In [77]:
train

,Sample_ID,1_elevation,2_elevation,3_elevation,4_elevation,5_elevation,6_elevation,7_elevation,8_elevation,9_elevation,...,17_sdoif,18_sdoif,19_sdoif,20_sdoif,21_sdoif,22_sdoif,23_sdoif,24_sdoif,25_sdoif,Label
0,1,130,129,127,126,123,126,125,124,122,...,1.281779,1.281743,1.281720,1.281684,1.281811,1.281788,1.281752,1.281729,1.281693,0
1,2,161,158,155,153,151,162,159,155,153,...,1.359639,1.359608,1.359587,1.359556,1.359683,1.359662,1.359631,1.359610,1.359579,1
2,3,149,151,154,156,158,154,157,158,160,...,1.365005,1.365025,1.365055,1.365075,1.364937,1.364967,1.364988,1.365018,1.365038,0
3,4,80,78,77,75,73,80,78,77,75,...,1.100708,1.100738,1.100759,1.100789,1.100630,1.100650,1.100680,1.100700,1.100731,0
4,5,117,115,114,112,110,115,113,111,110,...,1.284180,1.284130,1.284056,1.284006,1.284125,1.284050,1.284001,1.283926,1.283876,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10859,10860,203,202,201,200,199,202,201,201,199,...,1.271912,1.272011,1.272076,1.272174,1.271720,1.271786,1.271884,1.271950,1.272048,0
10860,10861,432,430,427,423,421,431,429,426,422,...,1.331153,1.331137,1.331113,1.331096,1.331207,1.331183,1.331167,1.331142,1.331126,1
10861,10862,348,345,342,339,335,344,342,339,336,...,1.298362,1.298337,1.298321,1.298297,1.298392,1.298376,1.298352,1.298336,1.298311,1
10862,10863,93,91,90,88,86,96,94,93,92,...,1.355035,1.355061,1.355099,1.355125,1.354937,1.354976,1.355002,1.355041,1.355067,0


In [78]:
id_=test["Sample_ID"]
test.drop("Sample_ID", axis=1, inplace=True)
train.drop("Sample_ID", axis=1, inplace=True)

In [79]:
col=train.columns[:-1]

In [80]:
col

Index(['1_elevation', '2_elevation', '3_elevation', '4_elevation',
       '5_elevation', '6_elevation', '7_elevation', '8_elevation',
       '9_elevation', '10_elevation',
       ...
       '16_sdoif', '17_sdoif', '18_sdoif', '19_sdoif', '20_sdoif', '21_sdoif',
       '22_sdoif', '23_sdoif', '24_sdoif', '25_sdoif'],
      dtype='object', length=225)

In [81]:
train1=pd.DataFrame()
test1=pd.DataFrame()

In [82]:
train["elevation_mean"]=train[train.columns[:25].to_list()].median(axis=1)
train["slope_mean"]=train[train.columns[25:50].to_list()].median(axis=1)
train["aspect_mean"]=train[train.columns[50:75].to_list()].median(axis=1)
train["placurv_mean"]=train[train.columns[75:100].to_list()].median(axis=1)
train["procurv_mean"]=train[train.columns[100:125].to_list()].median(axis=1)
train["lsfactor_mean"]=train[train.columns[125:150].to_list()].median(axis=1)
train["twi_mean"]=train[train.columns[150:175].to_list()].median(axis=1)
train["geology_mean"]=train[train.columns[175:200].to_list()].median(axis=1)
train["sdoif_mean"]=train[train.columns[200:225].to_list()].median(axis=1)
#################################################################
train["elevation_mean"]=train[train.columns[:25].to_list()].mean(axis=1)
train["slope_mean"]=train[train.columns[25:50].to_list()].mean(axis=1)
train["aspect_mean"]=train[train.columns[50:75].to_list()].mean(axis=1)
train["placurv_mean"]=train[train.columns[75:100].to_list()].mean(axis=1)
train["procurv_mean"]=train[train.columns[100:125].to_list()].mean(axis=1)
train["lsfactor_mean"]=train[train.columns[125:150].to_list()].mean(axis=1)
train["twi_mean"]=train[train.columns[150:175].to_list()].mean(axis=1)
train["geology_mean"]=train[train.columns[175:200].to_list()].mean(axis=1)
train["sdoif_mean"]=train[train.columns[200:225].to_list()].mean(axis=1)
########################################################################
train["elevation_count"]=train[train.columns[:25].to_list()].count(axis=1)
train["slope_count"]=train[train.columns[25:50].to_list()].count(axis=1)
train["aspect_count"]=train[train.columns[50:75].to_list()].count(axis=1)
train["placurv_count"]=train[train.columns[75:100].to_list()].count(axis=1)
train["procurv_count"]=train[train.columns[100:125].to_list()].count(axis=1)
train["lsfactor_count"]=train[train.columns[125:150].to_list()].count(axis=1)
train["twi_count"]=train[train.columns[150:175].to_list()].count(axis=1)
train["geology_count"]=train[train.columns[175:200].to_list()].count(axis=1)
train["sdoif_count"]=train[train.columns[200:225].to_list()].count(axis=1)
#########################################################################
train["elevation_std"]=train[train.columns[:25].to_list()].std(axis=1)
train["slope_std"]=train[train.columns[25:50].to_list()].std(axis=1)
train["aspect_std"]=train[train.columns[50:75].to_list()].std(axis=1)
train["placurv_std"]=train[train.columns[75:100].to_list()].std(axis=1)
train["procurv_std"]=train[train.columns[100:125].to_list()].std(axis=1)
train["lsfactor_std"]=train[train.columns[125:150].to_list()].std(axis=1)
train["twi_std"]=train[train.columns[150:175].to_list()].std(axis=1)
train["geology_std"]=train[train.columns[175:200].to_list()].std(axis=1)
train["sdoif_std"]=train[train.columns[200:225].to_list()].std(axis=1)
#########################################################################
train["elevation_min"]=train[train.columns[:25].to_list()].min(axis=1)
train["slope_min"]=train[train.columns[25:50].to_list()].min(axis=1)
train["aspect_min"]=train[train.columns[50:75].to_list()].min(axis=1)
train["placurv_min"]=train[train.columns[75:100].to_list()].min(axis=1)
train["procurv_min"]=train[train.columns[100:125].to_list()].min(axis=1)
train["lsfactor_min"]=train[train.columns[125:150].to_list()].min(axis=1)
train["twi_min"]=train[train.columns[150:175].to_list()].min(axis=1)
train["geology_min"]=train[train.columns[175:200].to_list()].min(axis=1)
train["sdoif_min"]=train[train.columns[200:225].to_list()].min(axis=1)
#########################################################################
train["elevation_max"]=train[train.columns[:25].to_list()].max(axis=1)
train["slope_max"]=train[train.columns[25:50].to_list()].max(axis=1)
train["aspect_max"]=train[train.columns[50:75].to_list()].max(axis=1)
train["placurv_max"]=train[train.columns[75:100].to_list()].max(axis=1)
train["procurv_max"]=train[train.columns[100:125].to_list()].max(axis=1)
train["lsfactor_max"]=train[train.columns[125:150].to_list()].max(axis=1)
train["twi_max"]=train[train.columns[150:175].to_list()].max(axis=1)
train["geology_max"]=train[train.columns[175:200].to_list()].max(axis=1)
train["sdoif_max"]=train[train.columns[200:225].to_list()].max(axis=1)
#########################################################################
train["elevation_kurt"]=train[train.columns[:25].to_list()].kurt(axis=1)
train["slope_kurt"]=train[train.columns[25:50].to_list()].kurt(axis=1)
train["aspect_kurt"]=train[train.columns[50:75].to_list()].kurt(axis=1)
train["placurv_kurt"]=train[train.columns[75:100].to_list()].kurt(axis=1)
train["procurv_kurt"]=train[train.columns[100:125].to_list()].kurt(axis=1)
train["lsfactor_kurt"]=train[train.columns[125:150].to_list()].kurt(axis=1)
train["twi_kurt"]=train[train.columns[150:175].to_list()].kurt(axis=1)
train["geology_kurt"]=train[train.columns[175:200].to_list()].kurt(axis=1)
train["sdoif_kurt"]=train[train.columns[200:225].to_list()].kurt(axis=1)
#########################################################################
train["elevation_skew"]=train[train.columns[:25].to_list()].skew(axis=1)
train["slope_skew"]=train[train.columns[25:50].to_list()].skew(axis=1)
train["aspect_skew"]=train[train.columns[50:75].to_list()].skew(axis=1)
train["placurv_skew"]=train[train.columns[75:100].to_list()].skew(axis=1)
train["procurv_skew"]=train[train.columns[100:125].to_list()].skew(axis=1)
train["lsfactor_skew"]=train[train.columns[125:150].to_list()].skew(axis=1)
train["twi_skew"]=train[train.columns[150:175].to_list()].skew(axis=1)
train["geology_skew"]=train[train.columns[175:200].to_list()].skew(axis=1)
train["sdoif_skew"]=train[train.columns[200:225].to_list()].skew(axis=1)
#########################################################################
train["elevation_q7.5"]=train[train.columns[:25].to_list()].quantile(q=0.75,axis=1)
train["slope_q7.5"]=train[train.columns[25:50].to_list()].quantile(q=0.75,axis=1)
train["aspect_q7.5"]=train[train.columns[50:75].to_list()].quantile(q=0.75,axis=1)
train["placurv_q7.5"]=train[train.columns[75:100].to_list()].quantile(q=0.75,axis=1)
train["procurv_q7.5"]=train[train.columns[100:125].to_list()].quantile(q=0.75,axis=1)
train["lsfactor_q7.5"]=train[train.columns[125:150].to_list()].quantile(q=0.75,axis=1)
train["twi_q7.5"]=train[train.columns[150:175].to_list()].quantile(q=0.75,axis=1)
train["geology_q7.5"]=train[train.columns[175:200].to_list()].quantile(q=0.75,axis=1)
train["sdoif_q7.5"]=train[train.columns[200:225].to_list()].quantile(q=0.75,axis=1)
#########################################################################
train["elevation_q5"]=train[train.columns[:25].to_list()].quantile(q=0.5,axis=1)
train["slope_q5"]=train[train.columns[25:50].to_list()].quantile(q=0.5,axis=1)
train["aspect_q5"]=train[train.columns[50:75].to_list()].quantile(q=0.5,axis=1)
train["placurv_q5"]=train[train.columns[75:100].to_list()].quantile(q=0.5,axis=1)
train["procurv_q5"]=train[train.columns[100:125].to_list()].quantile(q=0.5,axis=1)
train["lsfactor_q5"]=train[train.columns[125:150].to_list()].quantile(q=0.5,axis=1)
train["twi_q5"]=train[train.columns[150:175].to_list()].quantile(q=0.5,axis=1)
train["geology_q5"]=train[train.columns[175:200].to_list()].quantile(q=0.5,axis=1)
train["sdoif_q5"]=train[train.columns[200:225].to_list()].quantile(q=0.5,axis=1)
#########################################################################
train["elevation_q2.5"]=train[train.columns[:25].to_list()].quantile(q=0.25,axis=1)
train["slope_q2.5"]=train[train.columns[25:50].to_list()].quantile(q=0.25,axis=1)
train["aspect_q2.5"]=train[train.columns[50:75].to_list()].quantile(q=0.25,axis=1)
train["placurv_q2.5"]=train[train.columns[75:100].to_list()].quantile(q=0.25,axis=1)
train["procurv_q2.5"]=train[train.columns[100:125].to_list()].quantile(q=0.25,axis=1)
train["lsfactor_q2.5"]=train[train.columns[125:150].to_list()].quantile(q=0.25,axis=1)
train["twi_q2.5"]=train[train.columns[150:175].to_list()].quantile(q=0.25,axis=1)
train["geology_q2.5"]=train[train.columns[175:200].to_list()].quantile(q=0.25,axis=1)
train["sdoif_q2.5"]=train[train.columns[200:225].to_list()].quantile(q=0.25,axis=1)
#########################################################################
train.drop(col, axis=1, inplace=True)

In [83]:
test["elevation_mean"]=train[train.columns[:25].to_list()].median(axis=1)
test["slope_mean"]=train[train.columns[25:50].to_list()].median(axis=1)
test["aspect_mean"]=train[train.columns[50:75].to_list()].median(axis=1)
test["placurv_mean"]=train[train.columns[75:100].to_list()].median(axis=1)
test["procurv_mean"]=train[train.columns[100:125].to_list()].median(axis=1)
test["lsfactor_mean"]=train[train.columns[125:150].to_list()].median(axis=1)
test["twi_mean"]=train[train.columns[150:175].to_list()].median(axis=1)
test["geology_mean"]=train[train.columns[175:200].to_list()].median(axis=1)
test["sdoif_mean"]=train[train.columns[200:225].to_list()].median(axis=1)
#################################################################
test["elevation_mean"]=test[test.columns[:25].to_list()].mean(axis=1)
test["slope_mean"]=test[test.columns[25:50].to_list()].mean(axis=1)
test["aspect_mean"]=test[test.columns[50:75].to_list()].mean(axis=1)
test["placurv_mean"]=test[test.columns[75:100].to_list()].mean(axis=1)
test["procurv_mean"]=test[test.columns[100:125].to_list()].mean(axis=1)
test["lsfactor_mean"]=test[test.columns[125:150].to_list()].mean(axis=1)
test["twi_mean"]=test[test.columns[150:175].to_list()].mean(axis=1)
test["geology_mean"]=test[test.columns[175:200].to_list()].mean(axis=1)
test["sdoif_mean"]=test[test.columns[200:225].to_list()].mean(axis=1)
########################################################################
test["elevation_count"]=test[test.columns[:25].to_list()].count(axis=1)
test["slope_count"]=test[test.columns[25:50].to_list()].count(axis=1)
test["aspect_count"]=test[test.columns[50:75].to_list()].count(axis=1)
test["placurv_count"]=test[test.columns[75:100].to_list()].count(axis=1)
test["procurv_count"]=test[test.columns[100:125].to_list()].count(axis=1)
test["lsfactor_count"]=test[test.columns[125:150].to_list()].count(axis=1)
test["twi_count"]=test[test.columns[150:175].to_list()].count(axis=1)
test["geology_count"]=test[test.columns[175:200].to_list()].count(axis=1)
test["sdoif_count"]=test[test.columns[200:225].to_list()].count(axis=1)
#########################################################################
test["elevation_std"]=test[test.columns[:25].to_list()].std(axis=1)
test["slope_std"]=test[test.columns[25:50].to_list()].std(axis=1)
test["aspect_std"]=test[test.columns[50:75].to_list()].std(axis=1)
test["placurv_std"]=test[test.columns[75:100].to_list()].std(axis=1)
test["procurv_std"]=test[test.columns[100:125].to_list()].std(axis=1)
test["lsfactor_std"]=test[test.columns[125:150].to_list()].std(axis=1)
test["twi_std"]=test[test.columns[150:175].to_list()].std(axis=1)
test["geology_std"]=test[test.columns[175:200].to_list()].std(axis=1)
test["sdoif_std"]=test[test.columns[200:225].to_list()].std(axis=1)
#########################################################################
test["elevation_min"]=test[test.columns[:25].to_list()].min(axis=1)
test["slope_min"]=test[test.columns[25:50].to_list()].min(axis=1)
test["aspect_min"]=test[test.columns[50:75].to_list()].min(axis=1)
test["placurv_min"]=test[test.columns[75:100].to_list()].min(axis=1)
test["procurv_min"]=test[test.columns[100:125].to_list()].min(axis=1)
test["lsfactor_min"]=test[test.columns[125:150].to_list()].min(axis=1)
test["twi_min"]=test[test.columns[150:175].to_list()].min(axis=1)
test["geology_min"]=test[test.columns[175:200].to_list()].min(axis=1)
test["sdoif_min"]=test[test.columns[200:225].to_list()].min(axis=1)
#########################################################################
test["elevation_max"]=test[test.columns[:25].to_list()].max(axis=1)
test["slope_max"]=test[test.columns[25:50].to_list()].max(axis=1)
test["aspect_max"]=test[test.columns[50:75].to_list()].max(axis=1)
test["placurv_max"]=test[test.columns[75:100].to_list()].max(axis=1)
test["procurv_max"]=test[test.columns[100:125].to_list()].max(axis=1)
test["lsfactor_max"]=test[test.columns[125:150].to_list()].max(axis=1)
test["twi_max"]=test[test.columns[150:175].to_list()].max(axis=1)
test["geology_max"]=test[test.columns[175:200].to_list()].max(axis=1)
test["sdoif_max"]=test[test.columns[200:225].to_list()].max(axis=1)
#########################################################################
test["elevation_kurt"]=test[test.columns[:25].to_list()].kurt(axis=1)
test["slope_kurt"]=test[test.columns[25:50].to_list()].kurt(axis=1)
test["aspect_kurt"]=test[test.columns[50:75].to_list()].kurt(axis=1)
test["placurv_kurt"]=test[test.columns[75:100].to_list()].kurt(axis=1)
test["procurv_kurt"]=test[test.columns[100:125].to_list()].kurt(axis=1)
test["lsfactor_kurt"]=test[test.columns[125:150].to_list()].kurt(axis=1)
test["twi_kurt"]=test[test.columns[150:175].to_list()].kurt(axis=1)
test["geology_kurt"]=test[test.columns[175:200].to_list()].kurt(axis=1)
test["sdoif_kurt"]=test[test.columns[200:225].to_list()].kurt(axis=1)
#########################################################################
test["elevation_skew"]=test[test.columns[:25].to_list()].skew(axis=1)
test["slope_skew"]=test[test.columns[25:50].to_list()].skew(axis=1)
test["aspect_skew"]=test[test.columns[50:75].to_list()].skew(axis=1)
test["placurv_skew"]=test[test.columns[75:100].to_list()].skew(axis=1)
test["procurv_skew"]=test[test.columns[100:125].to_list()].skew(axis=1)
test["lsfactor_skew"]=test[test.columns[125:150].to_list()].skew(axis=1)
test["twi_skew"]=test[test.columns[150:175].to_list()].skew(axis=1)
test["geology_skew"]=test[test.columns[175:200].to_list()].skew(axis=1)
test["sdoif_skew"]=test[test.columns[200:225].to_list()].skew(axis=1)
#########################################################################
test["elevation_q7.5"]=test[test.columns[:25].to_list()].quantile(q=0.75,axis=1)
test["slope_q7.5"]=test[test.columns[25:50].to_list()].quantile(q=0.75,axis=1)
test["aspect_q7.5"]=test[test.columns[50:75].to_list()].quantile(q=0.75,axis=1)
test["placurv_q7.5"]=test[test.columns[75:100].to_list()].quantile(q=0.75,axis=1)
test["procurv_q7.5"]=test[test.columns[100:125].to_list()].quantile(q=0.75,axis=1)
test["lsfactor_q7.5"]=test[test.columns[125:150].to_list()].quantile(q=0.75,axis=1)
test["twi_q7.5"]=test[test.columns[150:175].to_list()].quantile(q=0.75,axis=1)
test["geology_q7.5"]=test[test.columns[175:200].to_list()].quantile(q=0.75,axis=1)
test["sdoif_q7.5"]=test[test.columns[200:225].to_list()].quantile(q=0.75,axis=1)
#########################################################################
test["elevation_q5"]=test[test.columns[:25].to_list()].quantile(q=0.5,axis=1)
test["slope_q5"]=test[test.columns[25:50].to_list()].quantile(q=0.5,axis=1)
test["aspect_q5"]=test[test.columns[50:75].to_list()].quantile(q=0.5,axis=1)
test["placurv_q5"]=test[test.columns[75:100].to_list()].quantile(q=0.5,axis=1)
test["procurv_q5"]=test[test.columns[100:125].to_list()].quantile(q=0.5,axis=1)
test["lsfactor_q5"]=test[test.columns[125:150].to_list()].quantile(q=0.5,axis=1)
test["twi_q5"]=test[test.columns[150:175].to_list()].quantile(q=0.5,axis=1)
test["geology_q5"]=test[test.columns[175:200].to_list()].quantile(q=0.5,axis=1)
test["sdoif_q5"]=test[test.columns[200:225].to_list()].quantile(q=0.5,axis=1)
#########################################################################
test["elevation_q2.5"]=test[test.columns[:25].to_list()].quantile(q=0.25,axis=1)
test["slope_q2.5"]=test[test.columns[25:50].to_list()].quantile(q=0.25,axis=1)
test["aspect_q2.5"]=test[test.columns[50:75].to_list()].quantile(q=0.25,axis=1)
test["placurv_q2.5"]=test[test.columns[75:100].to_list()].quantile(q=0.25,axis=1)
test["procurv_q2.5"]=test[test.columns[100:125].to_list()].quantile(q=0.25,axis=1)
test["lsfactor_q2.5"]=test[test.columns[125:150].to_list()].quantile(q=0.25,axis=1)
test["twi_q2.5"]=test[test.columns[150:175].to_list()].quantile(q=0.25,axis=1)
test["geology_q2.5"]=test[test.columns[175:200].to_list()].quantile(q=0.25,axis=1)
test["sdoif_q2.5"]=test[test.columns[200:225].to_list()].quantile(q=0.25,axis=1)
#########################################################################
test.drop(col, axis=1, inplace=True)

In [84]:
train.shape , test.shape

((10864, 91), (5430, 90))

In [85]:
y_train = train[['Label']]
df_train=train.drop(['Label'], axis=1)
cols=df_train.columns

In [86]:
# from sklearn.decomposition import PCA
# pca = PCA()
# df_train = pca.fit_transform(df_train)
# test = pca.fit_transform(test)
# df_train = df_train[:, :15]
# test = test[:, :15]
# df_train = pd.DataFrame(df_train, columns=train.columns[:15])
# test = pd.DataFrame(test, columns=train.columns[:15])

In [87]:
from sklearn.preprocessing import MinMaxScaler
ss = MinMaxScaler()
df_train = pd.DataFrame(ss.fit_transform(df_train), columns=df_train.columns)
test = pd.DataFrame(ss.transform(test), columns=test.columns)


In [88]:
# from imblearn.over_sampling import RandomOverSampler
# sampler = RandomOverSampler(random_state=11)
# from imblearn.over_sampling import SMOTE
# # sampler = SMOTE(k_neighbors=5, random_state=11)
# from imblearn.over_sampling import BorderlineSMOTE
# sampler = BorderlineSMOTE(k_neighbors=5, random_state=11)
from imblearn.under_sampling import RandomUnderSampler
sampler = RandomUnderSampler(random_state=11)

In [89]:
df_train.shape

(10864, 90)

In [90]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold,StratifiedKFold ,GroupKFold
from sklearn.metrics import mean_squared_error
from sklearn import  ensemble
def get_model(Name='lgbm') :
    if Name=='lgbm' :
      return LGBMClassifier(**{ 'boosting_type': 'gbdt','objective': 'binary','metric': 'auc',
                                'n_estimators': 500,'colsample_bytree' : 0.8,
                                'seed': 42,'silent':False,
                                'early_stopping_rounds': 100,
                                'learning_rate' : 0.05,'num_iterations': 1500,
                                'max_depth' :8 ,'num_leaves' : 150,
                                'max_bins': 85,'min_data_in_leaf':30 })
cols=test.columns
test_ = test[cols]
Model_Name = "lgbm"
folds = StratifiedKFold(n_splits=15, shuffle=True, random_state=5168)
oofs  = np.zeros((len(df_train[cols])))
test_predictions = np.zeros((len(test)))
tst_predd = []

for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train, train[['Label']])):
    X_train, y_train = df_train.iloc[trn_idx][cols], train.iloc[trn_idx]['Label']
    X_valid, y_valid = df_train.iloc[val_idx][cols], train.iloc[val_idx]['Label']
#     X_trn, y_trn = X.iloc[trn_idx], y.iloc[trn_idx]
#     X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
    X_train ,y_train = sampler.fit_resample(X_train ,y_train)
    from catboost import CatBoostClassifier

    cat_clf = CatBoostClassifier(subsample=0.36,
                            custom_loss='Logloss',
                           random_strength = 0,
                           max_depth=3,
                           eval_metric="AUC",
                           learning_rate=0.02,
                           iterations=2000,
                           bootstrap_type='Bernoulli',
                            l2_leaf_reg=0.3,
                           random_seed=432013,
                           od_type="Iter",
                           border_count=128
                           #has_time= True 
                          )
    from lightgbm import LGBMClassifier
    lightgbm = LGBMClassifier(
                       num_leaves=12,
                       learning_rate=0.05, 
                       n_estimators=5000,
                       max_bin=200, 
                       bagging_fraction=0.8,
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.5,
                       feature_fraction_seed=8,
                       min_sum_hessian_in_leaf = 11,
                       verbose=-1,
                       random_state=42)
#     clf = get_model(Name=Model_Name)
    clf= cat_clf
    model=lightgbm
    from sklearn.feature_selection import RFE
    from sklearn.pipeline import Pipeline

    rfe = RFE(estimator=cat_clf, n_features_to_select=200)
    model = lightgbm
    pipeline = Pipeline(steps=[('s',rfe),('m',model)])
    pipeline.fit(X_train, y_train)
    
    vp = pipeline.predict(X_valid)
    oofs[val_idx] = vp
    val_score = mean_squared_error((vp), (y_valid),squared=True)
    print(4*'-- -- -- --')
    print(f'Fold {fold_+1} Val score: {val_score}')
    print(4*'-- -- -- --')
    
    tp = pipeline.predict(test_)
    test_predictions += tp / folds.n_splits
    tst_predd.append(tp)

  
print()
print(3*'###',10*"^",3*'###')
print(mean_squared_error(train["Label"], oofs,squared=True))
print("Model training")
# clf.fit(X_train, y_train )

0:	total: 3.54ms	remaining: 7.07s
1:	total: 6.99ms	remaining: 6.98s
2:	total: 10.2ms	remaining: 6.81s
3:	total: 13.6ms	remaining: 6.76s
4:	total: 16.9ms	remaining: 6.75s
5:	total: 20.1ms	remaining: 6.69s
6:	total: 23.3ms	remaining: 6.64s
7:	total: 26.5ms	remaining: 6.59s
8:	total: 29.8ms	remaining: 6.59s
9:	total: 33.1ms	remaining: 6.58s
10:	total: 36.3ms	remaining: 6.56s
11:	total: 39.6ms	remaining: 6.56s
12:	total: 42.9ms	remaining: 6.55s
13:	total: 46.1ms	remaining: 6.54s
14:	total: 49.4ms	remaining: 6.53s
15:	total: 52.6ms	remaining: 6.52s
16:	total: 55.8ms	remaining: 6.51s
17:	total: 59ms	remaining: 6.5s
18:	total: 62.2ms	remaining: 6.49s
19:	total: 65.8ms	remaining: 6.51s
20:	total: 69.1ms	remaining: 6.51s
21:	total: 72.5ms	remaining: 6.52s
22:	total: 76.1ms	remaining: 6.54s
23:	total: 79.3ms	remaining: 6.53s
24:	total: 82.7ms	remaining: 6.53s
25:	total: 85.9ms	remaining: 6.52s
26:	total: 89ms	remaining: 6.5s
27:	total: 92.2ms	remaining: 6.49s
28:	total: 95.4ms	remaining: 6.49s
2

In [91]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold,StratifiedKFold ,GroupKFold
from sklearn.metrics import mean_squared_error
from sklearn import  ensemble
def get_model(Name='lgbm') :
    if Name=='lgbm' :
      return LGBMClassifier(**{ 'boosting_type': 'gbdt','objective': 'binary','metric': 'auc',
                                'n_estimators': 500,'colsample_bytree' : 0.8,
                                'seed': 42,'silent':False,
                                'early_stopping_rounds': 100,
                                'learning_rate' : 0.05,'num_iterations': 1500,
                                'max_depth' :8 ,'num_leaves' : 150,
                                'max_bins': 85,'min_data_in_leaf':30 })
cols=test.columns
test_ = test[cols]
Model_Name = "lgbm"
folds = StratifiedKFold(n_splits=15, shuffle=True, random_state=5168)
oofs  = np.zeros((len(df_train[cols])))
tstpred = np.zeros((len(test)))
tst_predd = []

for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train, train[['Label']])):
    X_train, y_train = df_train.iloc[trn_idx][cols], train.iloc[trn_idx]['Label']
    X_valid, y_valid = df_train.iloc[val_idx][cols], train.iloc[val_idx]['Label']
#     X_trn, y_trn = X.iloc[trn_idx], y.iloc[trn_idx]
#     X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
    X_train ,y_train = sampler.fit_resample(X_train ,y_train)
    
    clf = get_model(Name=Model_Name)
    clf.fit(X_train, y_train, eval_set = [(X_valid, y_valid)],
            verbose =100, early_stopping_rounds = 500)
    
    vp = clf.predict(X_valid)
    oofs[val_idx] = vp
    val_score = mean_squared_error((vp), (y_valid),squared=True)
    print(4*'-- -- -- --')
    print(f'Fold {fold_+1} Val score: {val_score}')
    print(4*'-- -- -- --')
    
    tp = clf.predict(test_)
    tstpred += tp / folds.n_splits
    tst_predd.append(tp)

  
print()
print(3*'###',10*"^",3*'###')
print(mean_squared_error(train["Label"], oofs,squared=True))
print("Model training")
# clf.fit(X_train, y_train )

[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] early_stopping_round is set=500, early_stopping_rounds=100 will be ignored. Current value: early_stopping_round=500
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Info] Number of positive: 2535, number of negative: 2535
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005367 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6238
[LightGBM] [Info] Number of data points in the train set: 5070, number of used features: 81
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] early_stopping_round is set=500, early_stopping_roun

In [92]:
from sklearn.metrics import classification_report
print(classification_report(train["Label"], oofs))

              precision    recall  f1-score   support

           0       0.94      0.80      0.86      8148
           1       0.58      0.85      0.69      2716

    accuracy                           0.81     10864
   macro avg       0.76      0.82      0.78     10864
weighted avg       0.85      0.81      0.82     10864



           precision    recall  f1-score   support

           0       0.89      0.94      0.91      8148
           1       0.77      0.66      0.71      2716

    accuracy                           0.87     10864
   macro avg       0.83      0.80      0.81     10864
weighted avg       0.86      0.87      0.86     10864

In [93]:
np.unique(test_predictions), np.unique(test_predictions)

(array([0.        , 0.06666667, 0.13333333, 0.2       , 0.26666667,
        0.33333333, 0.4       , 0.46666667, 0.53333333, 0.6       ,
        0.66666667, 0.73333333, 0.8       , 0.86666667, 0.93333333,
        1.        ]),
 array([0.        , 0.06666667, 0.13333333, 0.2       , 0.26666667,
        0.33333333, 0.4       , 0.46666667, 0.53333333, 0.6       ,
        0.66666667, 0.73333333, 0.8       , 0.86666667, 0.93333333,
        1.        ]))

In [94]:
sub=pd.DataFrame({"Sample_ID":id_})

In [95]:
sub['Label']=(np.round((test_predictions+tstpred+0.01)/2))
sub

,Sample_ID,Label
0,10865,0.0
1,10866,0.0
2,10867,0.0
3,10868,1.0
4,10869,1.0
...,...,...
5425,16290,0.0
5426,16291,1.0
5427,16292,0.0
5428,16293,0.0


In [96]:
# for i in range(len(test_predictions)):
#     if test_predictions[i]>=0.5:
#         test_predictions[i]=1
#     else:
#         test_predictions[i]=0

In [99]:
sub.Label.value_counts()

0.0    3185
1.0    2245
Name: Label, dtype: int64

In [100]:
from IPython.display import FileLink
def create_submission(submission_file, submission_name):
    submission_file.to_csv(submission_name+".csv",index=False)
    return FileLink(submission_name+".csv")
create_submission(sub, "sol")

/kaggle/working/sol.csv

#### 